In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0
train_images = train_images.reshape((train_images.shape[0], -1))  # Flatten images
test_images = test_images.reshape((test_images.shape[0], -1))

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)



11490434/11490434 [==============================] - 14s 1us/step


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

supervised_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_images.shape[1],)),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')  # 10 classes for digits 0-9
])

supervised_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

supervised_model.fit(train_images[:30000], train_labels[:30000], epochs=10, batch_size=128, validation_split=0.2)

supervised_model.save_weights('supervised_mnist_weights.h5')



Epoch 1/10


188/188 [==============================] - 7s 24ms/step - loss: 0.3444 - accuracy: 0.9038 - val_loss: 0.1709 - val_accuracy: 0.9450
Epoch 2/10
188/188 [==============================] - 4s 22ms/step - loss: 0.1232 - accuracy: 0.9644 - val_loss: 0.1428 - val_accuracy: 0.9555
Epoch 3/10
188/188 [==============================] - 4s 24ms/step - loss: 0.0797 - accuracy: 0.9753 - val_loss: 0.1223 - val_accuracy: 0.9633
Epoch 4/10
188/188 [==============================] - 4s 23ms/step - loss: 0.0486 - accuracy: 0.9853 - val_loss: 0.1043 - val_accuracy: 0.9683
Epoch 5/10
188/188 [==============================] - 5s 26ms/step - loss: 0.0306 - accuracy: 0.9903 - val_loss: 0.0995 - val_accuracy: 0.9703
Epoch 6/10
188/188 [==============================] - 4s 20ms/step - loss: 0.0215 - accuracy: 0.9935 - val_loss: 0.0992 - val_accuracy: 0.9725
Epoch 7/10
188/188 [==============================] - 4s 20ms/step - loss: 0.0128 - accuracy: 0.9965 - val_loss: 0.1113 - val_accuracy: 0.9

In [3]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(train_images, train_images, epochs=10, batch_size=256, shuffle=True, validation_data=(test_images, test_images))

encoder = Model(input_img, encoded)
encoder.save_weights('unsupervised_encoder_weights.h5')



Epoch 1/10
235/235 [==============================] - 10s 34ms/step - loss: 0.2119 - val_loss: 0.1311
Epoch 2/10
235/235 [==============================] - 6s 26ms/step - loss: 0.1150 - val_loss: 0.1012
Epoch 3/10
235/235 [==============================] - 5s 23ms/step - loss: 0.0952 - val_loss: 0.0884
Epoch 4/10
235/235 [==============================] - 6s 28ms/step - loss: 0.0852 - val_loss: 0.0809
Epoch 5/10
235/235 [==============================] - 8s 33ms/step - loss: 0.0794 - val_loss: 0.0766
Epoch 6/10
235/235 [==============================] - 7s 32ms/step - loss: 0.0759 - val_loss: 0.0740
Epoch 7/10
235/235 [==============================] - 7s 30ms/step - loss: 0.0736 - val_loss: 0.0721
Epoch 8/10
235/235 [==============================] - 6s 26ms/step - loss: 0.0721 - val_loss: 0.0709
Epoch 9/10
235/235 [==============================] - 6s 25ms/step - loss: 0.0709 - val_loss: 0.0700
Epoch 10/10
235/235 [==============================] - 6s 27ms/step - loss: 0.0701 - val_

In [4]:
supervised_model.load_weights('supervised_mnist_weights.h5')

encoder.load_weights('unsupervised_encoder_weights.h5')
for layer in encoder.layers:
    layer.trainable = False  # Freeze the encoder layers

final_model = Sequential(encoder.layers[:1])  # Using encoded representation
final_model.add(supervised_model)

final_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:
final_model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.2)

test_loss, test_acc = final_model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')


Epoch 1/10
375/375 [==============================] - 9s 21ms/step - loss: 0.0643 - accuracy: 0.9821 - val_loss: 0.0900 - val_accuracy: 0.9733
Epoch 2/10
375/375 [==============================] - 7s 20ms/step - loss: 0.0343 - accuracy: 0.9896 - val_loss: 0.0834 - val_accuracy: 0.9772
Epoch 3/10
375/375 [==============================] - 8s 20ms/step - loss: 0.0221 - accuracy: 0.9925 - val_loss: 0.1007 - val_accuracy: 0.9757
Epoch 4/10
375/375 [==============================] - 8s 22ms/step - loss: 0.0184 - accuracy: 0.9940 - val_loss: 0.0853 - val_accuracy: 0.9779
Epoch 5/10
375/375 [==============================] - 8s 22ms/step - loss: 0.0161 - accuracy: 0.9949 - val_loss: 0.1082 - val_accuracy: 0.9747
Epoch 6/10
375/375 [==============================] - 9s 23ms/step - loss: 0.0167 - accuracy: 0.9947 - val_loss: 0.1018 - val_accuracy: 0.9762
Epoch 7/10
375/375 [==============================] - 7s 19ms/step - loss: 0.0105 - accuracy: 0.9966 - val_loss: 0.1012 - val_accuracy: 0.9772